# Obtenemos la primera información de toda la región de la Araucania

In [1]:
import requests
import io
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader
import pandas as pd

## Obtendremos los enlaces y fechas para cada reporte

In [2]:
URL = 'https://seremi9.redsalud.gob.cl/accion-sanitaria-3/registros-covid19/'
page = requests.get(URL) #Obtenemos la respuesta del servidor

In [3]:
soup     = BeautifulSoup(page.content, 'html.parser')  #Transformamos a un objeto soup

In [4]:
job_elems = soup.find_all('ol')                    # Buscamos todos los elementos "ol"
for job_elem in job_elems:                         # Recorremos
    enlaces = job_elem.find_all('a')               # Buscamos los enlaces de ahí

In [5]:
reportes_diarios = []
base_url         = "https://seremi9.redsalud.gob.cl"
for enlace in enlaces:                            # Recorremos los enlaces
    fecha  = enlace.text                          # obtenemos el texto
    enlace = enlace['href']                       # obtenemos el enlace
    url    = base_url + enlace
    dato   = {'fecha': fecha, 'url':url}
    reportes_diarios.append(dato)

## Desde ahora trabajamos con los reportes diarios

In [6]:
#Esta funcion retornara en un arreglo el texto de la pagina 0 del PDF
def get_info(fecha, url):
    print ("Obteniendo información del: " + fecha)
    page      = requests.get(url)                           #Obtenemos la respuesta del servidor
    f         = io.BytesIO(page.content)                    #Tenemos el pdf
    reader    = PdfFileReader(f)                            #leemos el pf 
    contenido = reader.getPage(0).extractText().split('\n') #Tenemos el pdf separado por salto de linea
    
    
    #Filtramos e iteramos los datos
    pos_ini   = contenido.index('Angol')
    pos_fin   = contenido.index('Total General')
    
    contenido = contenido[pos_ini:pos_fin]                  # Filtramos con los datos que queremos    
    contenido.remove("Chol")
    contenido.remove("-")
    contenido = ["Chol-Chol" if x=="Chol" else x for x in contenido] #Cambiamos el nombre de cholchol
    
    #Eliminamos los espacios en blanco
    contenido_filtrado = []
    for c in contenido:
        if c != " ":
            contenido_filtrado.append(c)
    
    return contenido_filtrado    

In [7]:
# Retornará si el elemento ingresado es una ciudad
def is_city(elemento):
    bandera  = False
    ciudades = ['Angol', 'Carahue', 'Chol-Chol', 'Collipulli', 'Cunco', 'Curacautín', 'Curarrehue', 'Ercilla', 'Freire', 'Galvarino', 'Gorbea', 'Lautaro', 'Loncoche', 'Lonquimay', 'Los Sauces', 'Lumaco',  'Melipeuco', 'Nueva Imperial', 'Padre Las Casas',  'Perquenco', 'Pitrufquén', 'Pucón', 'Renaico',  'Saavedra', 'Temuco', 'Teodoro Schmidt','Toltén', 'Traiguén',  'Victoria', 'Vilcún', 'Villarrica', 'Purén']
    if elemento in ciudades:
        bandera = True
    return bandera

In [8]:
#Retornará los datos acumulados de los casos por region
def get_data(elemento, f):
    datos = [f]
    anterior = '' 
    ciudad   = ''
    n = 0
    for e in elemento:
        if (is_city(e) == False):
            anterior += e
        else:
            if anterior != '':
                #print (ciudad + " : " + anterior)
                datos.append(int(anterior))
                anterior = ''
        
        if is_city(e):
            ciudad = e
        
        ## Eliminamos el error del ultimo elemento
        if n == len(elemento)-1:
            datos.append(int(anterior))
        n += 1    
  
    return datos

La siguiente traerá los nombre de las ciudades que entrega el reporte diario.

In [9]:
def get_citys(fecha, url):
    ciudades = ['Fecha']
    data     = get_info(fecha, url)
    for d in data:
        if is_city(d):
            ciudades.append(d)
    return ciudades

ciudades = get_citys(reportes_diarios[0]['fecha'], reportes_diarios[0]['url'])

Obteniendo información del: 01 de Mayo


### Crearemos el dataframe de la región

In [10]:
araucania = pd.DataFrame(columns=ciudades)
araucania

,Fecha,Angol,Carahue,Chol-Chol,Collipulli,Cunco,Curacautín,Curarrehue,Ercilla,Freire,...,Purén,Renaico,Saavedra,Temuco,Teodoro Schmidt,Toltén,Traiguén,Victoria,Vilcún,Villarrica


### Llenamos de información el dataframe

In [11]:
for r in reportes_diarios:
    texto     = get_info(r['fecha'], r['url'])       # Obtenemos el texto de la primera pagina del informe
    data      = get_data(texto, r['fecha'])          # Obtenemos la data
    araucania.loc[len(araucania)] = data             # Creamos el data frame

Obteniendo información del: 01 de Mayo
Obteniendo información del: 02 de Mayo
Obteniendo información del: 03 de Mayo
Obteniendo información del: 04 de Mayo
Obteniendo información del: 05 de Mayo
Obteniendo información del: 06 de Mayo
Obteniendo información del: 07 de Mayo
Obteniendo información del: 08 de Mayo
Obteniendo información del: 09 de Mayo
Obteniendo información del: 10 de Mayo
Obteniendo información del: 11 de Mayo
Obteniendo información del: 12 de Mayo
Obteniendo información del: 13 de Mayo
Obteniendo información del: 14 de Mayo
Obteniendo información del: 15 de Mayo
Obteniendo información del: 16 de Mayo
Obteniendo información del: 17 de Mayo
Obteniendo información del: 18 de Mayo
Obteniendo información del: 19 de Mayo
Obteniendo información del: 20 de Mayo
Obteniendo información del: 21 de Mayo
Obteniendo información del: 22 de Mayo
Obteniendo información del: 23 de Mayo
Obteniendo información del: 24 de Mayo
Obteniendo información del: 25 de Mayo
Obteniendo información de

### Creamos el archivo csv

In [12]:
araucania.to_csv("output/araucania.csv", index=False)

# Trabajaremos con los casos nuevos

In [13]:
araucania.tail(5)

,Fecha,Angol,Carahue,Chol-Chol,Collipulli,Cunco,Curacautín,Curarrehue,Ercilla,Freire,...,Purén,Renaico,Saavedra,Temuco,Teodoro Schmidt,Toltén,Traiguén,Victoria,Vilcún,Villarrica
37,07 de Junio,180,69,18,32,39,8,4,16,81,...,8,26,90,839,10,16,18,118,93,34
38,08 de Junio,184,69,22,32,39,8,4,16,81,...,8,26,91,846,10,16,18,120,94,34
39,09 de Junio,190,68,22,33,40,9,4,16,83,...,8,26,92,848,10,16,19,120,94,34
40,10 de Junio,206,68,22,34,40,9,4,16,85,...,8,26,92,849,10,16,19,123,95,34
41,11 de Junio,214,68,22,34,45,10,4,16,87,...,8,26,92,852,10,16,19,125,98,37


### Realizamos la resta

In [14]:
comunas      = araucania.columns[1:]
nuevos_casos = araucania[comunas].diff()
nuevos_casos

,Angol,Carahue,Chol-Chol,Collipulli,Cunco,Curacautín,Curarrehue,Ercilla,Freire,Galvarino,...,Purén,Renaico,Saavedra,Temuco,Teodoro Schmidt,Toltén,Traiguén,Victoria,Vilcún,Villarrica
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,0,0,0,0,1,0,0,0,0,...,0,1,6,5,0,0,0,0,0,0
2,1,4,0,0,0,0,0,0,0,0,...,1,0,3,3,0,0,0,2,1,0
3,1,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,-2,0,0
4,2,0,0,0,0,0,0,0,0,1,...,1,2,4,6,0,0,0,1,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,3,1
6,2,1,0,0,0,0,0,0,0,0,...,0,1,0,7,0,0,0,1,1,0
7,0,0,0,0,1,1,0,0,0,0,...,0,2,4,8,0,0,0,1,0,0
8,2,0,0,0,0,0,0,0,0,0,...,1,0,3,4,0,0,0,0,2,0
9,2,0,0,0,0,0,0,0,2,0,...,0,0,5,16,0,0,0,0,0,0


### Agregamos la fecha nuevamente

In [15]:
nuevos_casos["Fecha"] = araucania["Fecha"].values

### Reodernamos las columnas y eliminamos la primera fila

In [16]:
cols = nuevos_casos.columns.to_list()
cols = cols[-1:] + cols[:-1]
nuevos_casos = nuevos_casos[cols]

In [17]:
nuevos_casos = nuevos_casos[1:]

In [18]:
nuevos_casos.tail(5)

,Fecha,Angol,Carahue,Chol-Chol,Collipulli,Cunco,Curacautín,Curarrehue,Ercilla,Freire,...,Purén,Renaico,Saavedra,Temuco,Teodoro Schmidt,Toltén,Traiguén,Victoria,Vilcún,Villarrica
37,07 de Junio,8,2,0,3,3,0,0,0,6,...,0,1,3,3,0,0,1,3,2,0
38,08 de Junio,4,0,4,0,0,0,0,0,0,...,0,0,1,7,0,0,0,2,1,0
39,09 de Junio,6,-1,0,1,1,1,0,0,2,...,0,0,1,2,0,0,1,0,0,0
40,10 de Junio,16,0,0,1,0,0,0,0,2,...,0,0,0,1,0,0,0,3,1,0
41,11 de Junio,8,0,0,0,5,1,0,0,2,...,0,0,0,3,0,0,0,2,3,3


### Creamos el archivo

In [19]:
nuevos_casos.to_csv("output/araucania_nuevoscasos.csv", index=False)